In [1]:
import json

# JSON 파일 경로
file_path_en = './learn_data/expanded_security_navigation_data_en.json'
file_path_kr = './learn_data/expanded_security_navigation_data_kr.json'

# JSON 파일 열기 및 데이터 로드
with open(file_path_en, 'r', encoding='utf-8') as f_en, open(file_path_kr, 'r', encoding='utf-8') as f_kr:
    data_en = json.load(f_en)
    data_kr = json.load(f_kr)

# 데이터 개수 확인
num_entries_en = len(data_en)
num_entries_kr = len(data_kr)

# 결과 출력
print(f'Number of entries in English dataset: {num_entries_en}')
print(f'Number of entries in Korean dataset: {num_entries_kr}')

Number of entries in English dataset: 2000
Number of entries in Korean dataset: 2000


In [2]:
import json

# JSON 파일 경로
file_path_en = './learn_data/informal_security_navigation_data_en.json'
file_path_kr = './learn_data/informal_security_navigation_data_kr.json'

# JSON 파일 열기 및 데이터 로드
with open(file_path_en, 'r', encoding='utf-8') as f_en, open(file_path_kr, 'r', encoding='utf-8') as f_kr:
    data_en = json.load(f_en)
    data_kr = json.load(f_kr)

# 데이터 개수 확인
num_entries_en = len(data_en)
num_entries_kr = len(data_kr)

# 결과 출력
print(f'Number of entries in English dataset: {num_entries_en}')
print(f'Number of entries in Korean dataset: {num_entries_kr}')


Number of entries in English dataset: 2000
Number of entries in Korean dataset: 2000


### 1. 환경 설정 및 데이터 로드

In [3]:
!pip install transformers datasets rouge-score nltk

import json
from datasets import load_dataset, Dataset, DatasetDict
from transformers import ElectraTokenizer, ElectraForSequenceClassification, Trainer, TrainingArguments, pipeline

# JSON 파일 경로
file_paths = [
    './learn_data/informal_security_navigation_data_en.json',
    './learn_data/informal_security_navigation_data_kr.json',
    './learn_data/expanded_security_navigation_data_en.json',
    './learn_data/expanded_security_navigation_data_kr.json',
    './learn_data/navigation_data_en.json',
    './learn_data/navigation_data_kr.json',
    './learn_data/security_data_en.json',
    './learn_data/security_data_kr.json'
]

# JSON 파일 열기 및 데이터 로드
data = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        data.extend(json.load(f))

print(f"총 데이터 개수: {len(data)}")


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using legacy 'setup.py install' for rouge-score, since package 'wheel' is not installed.
  Running setup.py install for rouge-score: started
  Running setup.py install for rouge-score: finished with status 'done'


You should consider upgrading via the 'C:\Users\Dongwook\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


총 데이터 개수: 10018


### 2. 데이터 전처리

In [4]:
# Tokenizer 및 모델 로드
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

# 데이터셋으로 변환
def convert_to_dataset(data):
    questions = [item['question'] for item in data]
    answers = [item['answer'] for item in data]
    labels = [0 if q == a else 1 for q, a in zip(questions, answers)]  # 임의의 레이블 생성
    dataset_dict = {'question': questions, 'answer': answers, 'labels': labels}
    return Dataset.from_dict(dataset_dict)

dataset = convert_to_dataset(data)

# DatasetDict 생성 (훈련, 검증 데이터셋 나누기)
split_dataset = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

# 데이터 전처리 함수
def preprocess_function(examples):
    return tokenizer(examples['question'], examples['answer'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

c:\Users\Dongwook\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dongwook\.cache\huggingface\hub\models--monologg--koelectra-base-v3-discriminator. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Map:   0%|          | 0/9016 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9016
    })
    validation: Dataset({
        features: ['question', 'answer', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1002
    })
})


### 3. 학습 설정


In [5]:
from transformers import ElectraForSequenceClassification, Trainer, TrainingArguments, get_linear_schedule_with_warmup
from tqdm import tqdm
import torch
import time

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.progress_bar = None

    def create_optimizer_and_scheduler(self, num_training_steps: int):
        super().create_optimizer_and_scheduler(num_training_steps)
        self.progress_bar = tqdm(total=num_training_steps, desc="Training Progress")

    def training_step(self, model, inputs):
        loss = super().training_step(model, inputs)
        self.progress_bar.update(1)
        elapsed_time = time.time() - self.start_time
        steps_per_second = self.state.global_step / elapsed_time if elapsed_time > 0 else float('inf')
        remaining_time = (self.state.max_steps - self.state.global_step) / steps_per_second if steps_per_second > 0 else float('inf')
        self.progress_bar.set_postfix({
            "elapsed_time": f"{elapsed_time // 3600:.0f}h {elapsed_time % 3600 // 60:.0f}m {elapsed_time % 60:.0f}s",
            "remaining_time": f"{remaining_time // 3600:.0f}h {remaining_time % 3600 // 60:.0f}m {remaining_time % 60:.0f}s" if remaining_time != float('inf') else "Calculating...",
            "loss": loss.item()
        })
        return loss

    def _save(self, output_dir: str, state_dict=None):
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()
        self.model.save_pretrained(output_dir, state_dict=state_dict, safe_serialization=self.args.save_safetensors)
        if self.tokenizer is not None:
            self.tokenizer.save_pretrained(output_dir)

    def train(self, *args, **kwargs):
        self.start_time = time.time()
        super().train(*args, **kwargs)
        self.progress_bar.close()

# 모델 로드
model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator')

# 학습 설정
training_args = TrainingArguments(
    output_dir='./koelectra2',             # before: './koelectra_results', after: './koelectra2'
    eval_strategy="epoch",                 # before: "evaluation_strategy", after: "eval_strategy"
    save_strategy="epoch",                 # before: "epoch", after: "epoch" (unchanged)
    learning_rate=3e-5,                    # before: 2e-5, after: 3e-5 (학습률 증가)
    per_device_train_batch_size=32,        # before: 16, after: 32 (배치 크기 증가)
    per_device_eval_batch_size=32,         # before: 16, after: 32 (배치 크기 증가)
    num_train_epochs=3,                    # before: 1, after: 3 (에포크 수 증가)
    weight_decay=0.01,                     # before: 0.01, after: 0.01 (unchanged)
    logging_dir='./logs',                  # before: './logs', after: './logs' (unchanged)
    logging_steps=200,                     # before: 500, after: 200 (로그 기록 빈도 증가)
    save_total_limit=3,                    # before: 2, after: 3 (저장 제한 수 증가)
    fp16=True                              # 추가: 혼합 정밀도 훈련 활성화
)

# 학습률 스케줄러 추가
total_steps = len(tokenized_datasets['train']) // training_args.per_device_train_batch_size * training_args.num_train_epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps * 0.1,  # 초기 10% 워밍업 단계
    num_training_steps=total_steps
)

# CustomTrainer 설정
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    optimizers=(optimizer, scheduler)  # 최적화기와 스케줄러 추가
)



pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4. 모델 학습

In [6]:
def train_model():
    # 학습 시작 시간 기록
    trainer.start_time = time.time()

    # 모델 학습
    trainer.train()

    # 학습 완료 시간 기록
    end_time = time.time()

    # 학습 시간 계산
    training_time = end_time - trainer.start_time
    hours, rem = divmod(training_time, 3600)
    minutes, seconds = divmod(rem, 60)

    # 결과 출력
    print(f'Training completed in {int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds')

train_model()

Training Progress:   0%|          | 0/846 [00:00<?, ?it/s]

  0%|          | 0/846 [00:00<?, ?it/s]

Training Progress:  24%|██▎       | 200/846 [6:59:21<22:41:28, 126.45s/it, elapsed_time=6h 59m 19s, remaining_time=22h 43m 19s, loss=0.0017] 

{'loss': 0.1126, 'grad_norm': 0.03061777912080288, 'learning_rate': 2.542506919731119e-05, 'epoch': 0.71}


Training Progress:  33%|███▎      | 282/846 [9:52:08<18:09:50, 115.94s/it, elapsed_time=9h 52m 9s, remaining_time=19h 50m 37s, loss=0.00118]  

...

  0%|          | 0/32 [00:00<?, ?it/s]

Training Progress: 100%|██████████| 846/846 [30:45:20<00:00, 116.01s/it, elapsed_time=30h 29m 38s, remaining_time=0h 2m 10s, loss=0.000267]

{'eval_loss': 0.00014077186642680317, 'eval_runtime': 937.9464, 'eval_samples_per_second': 1.068, 'eval_steps_per_second': 0.034, 'epoch': 3.0}


Training Progress: 100%|██████████| 846/846 [30:45:21<00:00, 130.88s/it, elapsed_time=30h 29m 38s, remaining_time=0h 2m 10s, loss=0.000267]

{'train_runtime': 110721.1528, 'train_samples_per_second': 0.244, 'train_steps_per_second': 0.008, 'train_loss': 0.027017994616488898, 'epoch': 3.0}
Training completed in 30 hours, 45 minutes and 21 seconds


### 5. 모델 저장

In [7]:
trainer.save_model("./koelectra2")

### 6. 모델 평가

In [8]:
# 모델 평가
results = trainer.evaluate()

# 평가 결과 출력
print("Evaluation Results:", results)


  0%|          | 0/32 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 0.00014077186642680317, 'eval_runtime': 933.503, 'eval_samples_per_second': 1.073, 'eval_steps_per_second': 0.034, 'epoch': 3.0}


### 7. 모델 정보 출력

In [9]:
# 모델 정보와 특징
model_info = {
    "Model": "KoELECTRA",
    "Description": "A lightweight and efficient model optimized for Korean natural language processing.",
    "Base Model": "ELECTRA",
    "Publisher": "KakaoBrain",
    "Advantages": [
        "Efficient and fast inference",
        "Good performance with limited resources",
        "Optimized for Korean text classification and sentiment analysis"
    ]
}

print(json.dumps(model_info, indent=4))


{
    "Model": "KoELECTRA",
    "Description": "A lightweight and efficient model optimized for Korean natural language processing.",
    "Base Model": "ELECTRA",
    "Publisher": "KakaoBrain",
    "Advantages": [
        "Efficient and fast inference",
        "Good performance with limited resources",
        "Optimized for Korean text classification and sentiment analysis"
    ]
}


### 8. 최적화 - 동적 양자화

In [11]:
import torch
from transformers import ElectraForSequenceClassification

# 모델 로드
model_fp32 = ElectraForSequenceClassification.from_pretrained("./koelectra2")

# 동적 양자화 적용
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)

# 최적화된 모델 저장
torch.save(model_int8.state_dict(), "./koelectra2_quantized/pytorch_model.bin")
print("Model quantized and saved.")

Model quantized and saved.


1. 동적 양자화 (Dynamic Quantization)
- **특징**: 런타임 시 활성화 값을 양자화하며, 모델 가중치는 정밀도를 유지.
- **장점**: 
  - 빠르고 간단하게 적용 가능
  - CPU 기반 추론 속도 향상
  - 메모리 사용량 감소
- **단점**: 
  - 정적 양자화나 QAT보다 정확도가 다소 떨어질 수 있음
- **적합한 상황**: 
  - 모델이 이미 훈련된 상태에서 재훈련 없이 최적화를 원할 때
  - 웹 및 모바일 환경에서 CPU 기반 추론 속도를 높이고자 할 때

2. 정적 양자화 (Static Quantization)
- **특징**: 모델 가중치와 활성화 값을 양자화하며, 양자화하기 전에 일부 데이터로 모델을 '준비'.
- **장점**: 
  - 더 높은 정확도를 제공
  - 런타임 성능이 우수
  - 메모리 사용량과 추론 속도 모두 최적화
- **단점**: 
  - 일부 데이터를 사용하여 모델을 '준비'하는 추가 단계 필요
- **적합한 상황**: 
  - 추론 시 정확도와 성능 모두를 개선하고자 할 때
  - 데이터 샘플을 사용하여 모델을 준비할 수 있을 때

3. 지연 양자화 (Quantization-Aware Training, QAT)
- **특징**: 양자화된 모델을 훈련하여 정확도를 유지함.
- **장점**: 
  - 양자화로 인한 성능 손실 최소화
  - 높은 정확도 유지
- **단점**: 
  - 전체 훈련 파이프라인을 다시 실행해야 함
  - 훈련 시간이 길어질 수 있음
- **적합한 상황**: 
  - 양자화로 인한 성능 저하를 최소화하고 정확도를 유지하면서 최적화를 하고자 할 때
  - 훈련에 필요한 리소스와 시간이 충분할 때

4. 프루닝 (Pruning)
- **특징**: 모델의 파라미터 수를 줄여 경량화함.
- **장점**: 
  - 모델의 크기와 메모리 사용량 감소
  - 특정 기준에 따라 중요도가 낮은 파라미터를 제거하여 모델을 최적화
- **단점**: 
  - 프루닝 후 재훈련 필요
  - 잘못된 프루닝으로 인해 성능 저하 가능
- **적합한 상황**: 
  - 모델의 크기를 줄이고 메모리 사용량을 줄이고자 할 때
  - 프루닝 후 재훈련을 통해 모델 성능을 유지할 수 있을 때

5. 혼합 정밀도 훈련 (Mixed Precision Training)
- **특징**: 모델의 일부 파라미터를 반 정밀도로 훈련하여 메모리 사용량을 줄이고 훈련 속도를 높임.
- **장점**: 
  - 훈련 속도 향상
  - 메모리 사용량 감소
- **단점**: 
  - 모든 하드웨어에서 지원되지 않을 수 있음
  - 모델의 안정성에 영향을 미칠 수 있음
- **적합한 상황**: 
  - 훈련 시간이 긴 대형 모델의 훈련 속도를 높이고자 할 때
  - GPU를 활용하여 훈련하는 경우


### 9. 시뮬레이션

In [12]:
from transformers import pipeline

# 정량화된 모델 로드
quantized_model = ElectraForSequenceClassification.from_pretrained("./koelectra2_quantized")
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

# 파이프라인 생성
nlp = pipeline("text-classification", model=quantized_model, tokenizer=tokenizer)

# 샘플 텍스트로 시뮬레이션 수행
sample_texts = [
    "이 영화 정말 재미있어요!",
    "서비스가 별로였어요.",
    "가격 대비 품질이 훌륭합니다."
]

# 예측 수행 및 결과 출력
for text in sample_texts:
    result = nlp(text)
    print(f"Text: {text}\nPrediction: {result}\n")


c:\Users\Dongwook\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:383: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,
Some weights of the model checkpoint at ./koelectra2_quantized were not used when initializing ElectraForSequenceClassification: ['classifier.dense._packed_params._packed_params', 'classifier.dense._packed_params.dtype', 'classifier.dense.scale', 'classifier.dense.zero_point', 'classifier.out_proj._packed_params._packed_params', 'classifier.out_proj._packed_params.dtype', 'classifier.out_proj.scale', 'classifier.out_proj.zero_point', 'electra.encoder.layer.0.attention.output.dense._packed_params._packed_params', 'electra.encoder.layer.0.attention.output.dense._packed_params.dtype', 'elec

Text: 이 영화 정말 재미있어요!
Prediction: [{'label': 'LABEL_0', 'score': 0.5522649884223938}]

Text: 서비스가 별로였어요.
Prediction: [{'label': 'LABEL_0', 'score': 0.5521101951599121}]

Text: 가격 대비 품질이 훌륭합니다.
Prediction: [{'label': 'LABEL_0', 'score': 0.5509616136550903}]



### 10. 평가

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 평가 데이터에 대한 예측 수행
predictions = trainer.predict(tokenized_datasets['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(axis=1)

# 정확도 계산
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# 정밀도 계산
precision = precision_score(y_true, y_pred, average='weighted')
print(f'Precision: {precision:.4f}')

# 재현율 계산
recall = recall_score(y_true, y_pred, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 점수 계산
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 Score: {f1:.4f}')

# 혼동 행렬 계산
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)


  0%|          | 0/32 [00:00<?, ?it/s]

1. BLEU (Bilingual Evaluation Understudy) Score:

- 기계 번역 및 텍스트 생성 모델의 품질을 평가하기 위해 사용되는 지표입니다.
- 모델이 생성한 텍스트와 참조 텍스트 간의 n-gram 일치를 측정합니다.

2. ROUGE (Recall-Oriented Understudy for Gisting Evaluation) Score:
- 텍스트 요약 및 생성 모델의 품질을 평가하기 위해 사용됩니다.
- ROUGE-N, ROUGE-L 등의 변형이 있으며, 주로 n-gram, LCS(Longest Common Subsequence)을 기반으로 측정합니다.

3. Perplexity:
- 언어 모델의 예측 성능을 평가하는 지표입니다.
- 낮을수록 모델이 테스트 데이터의 분포를 더 잘 예측한다는 것을 의미합니다.

In [ ]:
from datasets import load_metric

# BLEU 및 ROUGE 메트릭 로드
bleu = load_metric('bleu')
rouge = load_metric('rouge')

# 실제 응답과 모델 응답 예시
references = [
    ["this is a test", "this is a trial"],
    ["another test here"]
]
predictions = [
    "this is a test",
    "another test"
]

# BLEU 점수 계산
bleu_result = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_result['bleu']}")

# ROUGE 점수 계산
rouge_result = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE Score: {rouge_result}")


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

# 토크나이저와 모델 로드
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
model = ElectraForSequenceClassification.from_pretrained('./koelectra2')

# 테스트 데이터 예시
test_texts = [
    "이 영화 정말 재미있어요!",
    "서비스가 별로였어요."
]

# 텍스트 토크나이즈
inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True)

# 모델 예측 수행
with torch.no_grad():
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

# Perplexity 계산
perplexity = torch.exp(loss)
print(f"Perplexity: {perplexity.item()}")
